# Training Pipeline Example

## 1. Imports

In [19]:
%reload_ext dotenv
%dotenv

import warnings
warnings.filterwarnings(action='ignore', module='.*paramiko.*')

import os

from src.training_pipeline import train_pipeline
import io

from src.types.hyperparameters import HyperParameters
from src.types.model_metadata import ModelMetadata

## 2. Defining model parameters and reward function

In [20]:
# Default values set from official documentation
model_name = 'rl-deepracer-sagemaker'
hyperparameters = HyperParameters()
model_metadata = ModelMetadata()

In [21]:
hyperparameters

HyperParameters(batch_size=64, beta_entropy=0.01, discount_factor=0.999, e_greedy_value=0.05, epsilon_steps=10000, exploration_type=<ExplorationType.CATEGORICAL: 'categorical'>, loss_type=<LossType.HUBER: 'huber'>, lr=0.0003, num_episodes_between_training=40, num_epochs=3, stack_size=1, term_cond_avg_score=100000, term_cond_max_episodes=100000)

In [22]:
model_metadata

ModelMetadata(action_space_type=<ActionSpaceType.CONTINUOUS: 'continuous'>, action_space=ContinuousActionSpace(steering_angle=SteeringAngle(high=30.0, low=-30.0), speed=Speed(high=4.0, low=1.0)), version=5, training_algorithm=<TrainingAlgorithm.PPO: 'ppo'>, neural_network=<NeuralNetwork.DEEP_CONVOLUTIONAL_NETWORK_SHALLOW: 'DEEP_CONVOLUTIONAL_NETWORK_SHALLOW'>, sensor=[<Sensor.FRONT_FACING_CAMERA: 'FRONT_FACING_CAMERA'>])

In [23]:
def reward_function(params):
  reward = 1.0
  return float(reward)

## 3. Pipelines

### Training

In [26]:
train_pipeline(
  model_name=model_name,
  hyperparameters=hyperparameters,
  model_metadata=model_metadata,
  reward_function=reward_function,
  overwrite=False
)

Data uploaded successfully to custom files
The reward function copied successfully to models folder at rl-deepracer-sagemaker/reward_function.py
Upload successfully the RoboMaker training configurations
Exposing the envs from config.env and system.env
Starting model training
['/Users/jv/Desktop/uni/drfc-manager/config/drfc-images/docker-compose-training.yml', '/Users/jv/Desktop/uni/drfc-manager/config/drfc-images/docker-compose-keys.yml', '/Users/jv/Desktop/uni/drfc-manager/config/drfc-images/docker-compose-endpoint.yml']


time="2025-04-10T20:33:58-03:00" level=warning msg="The \"DR_ROBOMAKER_TRAIN_PORT\" variable is not set. Defaulting to a blank string."
time="2025-04-10T20:33:58-03:00" level=warning msg="The \"DR_ROBOMAKER_GUI_PORT\" variable is not set. Defaulting to a blank string."
time="2025-04-10T20:33:58-03:00" level=warning msg="/Users/jv/Desktop/uni/drfc-manager/config/drfc-images/docker-compose-training.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
time="2025-04-10T20:33:58-03:00" level=warning msg="/Users/jv/Desktop/uni/drfc-manager/config/drfc-images/docker-compose-keys.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
time="2025-04-10T20:33:58-03:00" level=warning msg="/Users/jv/Desktop/uni/drfc-manager/config/drfc-images/docker-compose-endpoint.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 Container drfc-i

Attaching to rl_coach-1, robomaker-1
rl_coach-1   | sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
rl_coach-1   | sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
rl_coach-1   | Model checkpoints and other metadata will be stored at: s3://tcc-experiments/rl-deepracer-sagemaker
rl_coach-1   | Uploading to s3://tcc-experiments/rl-deepracer-sagemaker
rl_coach-1   | Using image awsdeepracercommunity/deepracer-simapp:5.3.3-gpu


rl_coach-1   | Traceback (most recent call last):
rl_coach-1   |   File "/opt/code/ml/rl_coach/start.py", line 99, in <module>
rl_coach-1   |     s3Client.download_fileobj(s3_bucket, hyperparameter_file, hyperparameter_data)
rl_coach-1   |   File "/usr/local/lib/python3.8/dist-packages/boto3/s3/inject.py", line 801, in download_fileobj
rl_coach-1   |     return future.result()
rl_coach-1   |   File "/usr/local/lib/python3.8/dist-packages/s3transfer/futures.py", line 103, in result
rl_coach-1   |     return self._coordinator.result()
rl_coach-1   |   File "/usr/local/lib/python3.8/dist-packages/s3transfer/futures.py", line 264, in result
rl_coach-1   |     raise self._exception
rl_coach-1   |   File "/usr/local/lib/python3.8/dist-packages/s3transfer/tasks.py", line 265, in _main
rl_coach-1   |     self._submit(transfer_future=transfer_future, **kwargs)
rl_coach-1   |   File "/usr/local/lib/python3.8/dist-packages/s3transfer/download.py", line 352, in _submit
rl_coach-1   |     response 

rl_coach-1 exited with code 1
robomaker-1 exited with code 0
